In [1]:
#Importar librerías para tratamiento de datos
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
import re
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

from selenium.common.exceptions import NoSuchElementException



In [13]:

######BUENO####
df_pelis = pd.read_csv("peliculas_api.csv") 
df_pelis.keys()
id_IMDB= df_pelis["ID de la Película"]


# Carga archivo patri
df_pelis = pd.read_csv("peliculas_api.csv")

# los ID son string
df_pelis["ID de la Película"] = df_pelis["ID de la Película"].astype(str)

# abre google
driver = webdriver.Chrome()

# Itera sobre los ID
for id_imdb in df_pelis["ID de la Película"][:1]:
    try:
        # carga la página de IMDB
        url = f"https://www.imdb.com/title/{id_imdb}/"
        driver.get(url)

        sleep(3)

         # acepta las cookies
        try:
            driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
        except NoSuchElementException:
            pass   

        
        driver.maximize_window()

        # Haz clic en el enlace "Reparto y equipo"
        driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button').click()
        sleep(2)

         #Voy a reparto y equipo
        driver.find_element("xpath", "/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a").click()
        sleep(2)

        # Inicializa un diccionario para almacenar los datos de los actores
        dicc_actors = {
            "pelicula":[],
            "Nombre actr": [],
            "conocide_por": [],
            "año_nacimiento": [],
            "profesión": [],
            "premios": []
        }

        # itera sobre los 10 primeros actores
        for i in range(2, 12):
            try:
               
               #nombre pelicula
                pelicula= driver.find_element("css selector","#main > div.article.listo > div.subpage_title_block > div > div.parent > h3 > a").text
                dicc_actors["pelicula"].append(pelicula)
                
                # Haz clic en el enlace del actor
                driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
                sleep(3)

                # nombre actr
                #nombre_act = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div > h1 > span").text
                nombre_act = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div > h1 > span").text
                dicc_actors["Nombre actr"].append(nombre_act)

                sleep(2)

                # películas por las que se le conoce
                #conocide_por= driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a").text
                conocide_por= driver.find_element("xpath" '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[2]/section[1]/div[2]/div/div[2]/div[1]/div[2]/div[1]/a').text
                dicc_actors["conocide_por"].append(conocide_por)

                sleep(2)
                try:
                  #conocide_por2=driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(2) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a").text
                  conocide_por2= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[2]/section[1]/div[2]/div/div[2]/div[2]/div[2]/div[1]/a').text
                  dicc_actors["conocide_por"].append(conocide_por2)
                except NoSuchElementException:
                  pass

                sleep(2)

                # año de nacimiento del actor
                
                
                #año_nacimiento= driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-4.iDcoFI > div.sc-92625f35-6.gHKhNg > div.sc-92625f35-11.eyxQlE > section > aside > div > span:nth-child(2)").text
                año_nacimiento= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/section/aside/div/span[2]').text
                dicc_actors["año_nacimiento"].append(año_nacimiento)
            
                        
                    #año_nacimiento= driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-4.brwmyD > div > div.sc-92625f35-11.qOBXf > section > aside > div > span:nth-child(2)")

                # la profesión del actor
                #profesion = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div > ul").text
                profesion=driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text
                dicc_actors["profesión"].append(profesion)

                # los premios del actor, si los tiene
                try:
                    #premios = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul/li/span').text   #"css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(8) > div > ul > li > div > ul > li > span").text
                    premios = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul/li/span').text
                    dicc_actors["premios"].append(premios)

                except NoSuchElementException:
                    dicc_actors["premios"].append("N/A")

            except NoSuchElementException:
                pass  # No se encontró el enlace del actor, continuar con el siguiente

            finally:
                # Regresa a la página anterior para seleccionar al siguiente actor
                driver.back()
                sleep(2)

        # Imprime los datos de los actores para esta película
        print(f"Datos de los actores para la película con ID de IMDb {id_imdb}:")
        print(dicc_actors)
    
    except Exception as e:
        print(f"Error al procesar la película con ID de IMDb {id_imdb}: {e}")

# Cierra el WebDriver al finalizar
driver.quit()


#cambio ficticio

In [ ]:
df2 = pd.DataFrame(dicc_actors)
print(df2)
